# Image Formation Module: Compression

**Notes from Hany Farid's Computer Vision Course (UC Berkley)**
https://farid.berkeley.edu/downloads/tutorials/learnComputerVision/

In [7]:
## Packages we'll need for our exercises below
import numpy as np

## [JPEG Compression](https://farid.berkeley.edu/learnComputerVision/lectures/cv-02-14-JPEGcompression.mp4)

JPEG storage is cheap but uses lossy compression to reduce the size of the image - the general process is:

1. RGB (in [0,255])
2. RGB to YCbCr (in [0,255])
3. Downsample Cb/Cr (optional)
4. Partition each channel into non-overlapping 8x8 pixel blocks
5. Discrete cosine transform (DCT-II) each block
6. Quantize DCT coefficients
7. Entropy encode quantized DCT coefficients

To execute the discrete cosine transform you need to take each value $u$ (rows) and $v$ (cols) and compute the following:

$ F_c(u,v) = \alpha _{u,v} 
\sum_{x=0} ^7
\sum_{y=0} ^7
f_c(x,y) 
\cos(\frac{(2x+1)u\pi} {16})
\cos(\frac{(2y+1)v\pi} {16})
$

To then quantize the DCT coefficents you need to compute:

$\hat{F}_c(u,v)
\lfloor \frac{F_c(u,v)}{q_c(u,v)} \rfloor$

Finding the DCT doesn't save any space - but the savings comes from quantizing the values because it takes each value and divides by some number then finds the floor... so all the small values can be encoded as 0

### Exercise #5 - Compute quantized DCT coefficients

> Write a Python function that takes as input an 8x8 numpy array of random pixel values and an 8x8 numpy array of quantization values, and returns the quantized 8x8 DCT coefficients.

$ F_c(u,v) = \alpha _{u,v} 
\sum_{x=0} ^7
\sum_{y=0} ^7
f_c(x,y) 
\cos(\frac{(2x+1)u\pi} {16})
\cos(\frac{(2y+1)v\pi} {16})
$

Where:
$\alpha_u,v = \alpha_u \alpha_v$

if $u = 0$ then $\alpha_u, \alpha_v$ = $\sqrt{1/8}$ else $\sqrt{2/8}$

--- 
Hany helps us break down the DCT equation in setting up this exercise;

- $ F_c(u,v)$ = the DCT transform where u and v go from 0-7 across and down in our 8x8 matrix below

- $ \alpha _{u,v} $ is explained above - it's just a scaling value 

- $ \sum_{x=0} ^7
\sum_{y=0} ^7 $ means that we're doing a double sum over all values of x and y

- $ f_c(x,y) $ are the actual x,y in each loop as we iterate through the double sum above

- $ \cos(\frac{(2x+1)u\pi} {16})
\cos(\frac{(2y+1)v\pi} {16}) $ for each x,y and u,v we're finding the cosine terms defined here and summing them all up

This will give us the DCT coeffient for $(u,v)$



In [8]:
# array of random pixel values
P = np.random.randint(0, 255, size=(8, 8))

# array of quantization values (setting all to two for now)
Q = np.empty(shape=(8,8))
Q.fill(2)

In [9]:
def findDCT(P, Q):
    if (P.shape[0] != P.shape[1]) | (len(P.shape) != 2) | (P.shape != Q.shape):
        raise('assumes square matrices as input')

    s = P.shape[0]
    DCT = np.zeros(shape=(s,s))

    # Loop over every u,v as row/cols in the input matrix P
    for u in range(0, s):
        for v in range(0, s):
            au = np.sqrt(1/8) if u == 0 else np.sqrt(2/8)
            av = np.sqrt(1/8) if v == 0 else np.sqrt(2/8)
            
            # double sum over every x,y value in pixel matrix P
            s2 = 0
            for x in range(0,8):
                s1 = 0
                for y in range(0,8):
                    s1 += P[x,y] * np.cos(((2*x+1)*u*np.pi)/16) * np.cos(((2*y+1)*v*np.pi)/16)
                s2 += s1
            DCT[u,v] = au * av * s2

    return np.array( DCT/Q, dtype=int )

print('input pixel matrix: \n', P, '\n')
print('quantized DCT coefficents: \n', findDCT(P,Q))
    
    

input pixel matrix: 
 [[  7 194 227  80 114  76  96  79]
 [108 160 202  20 104 107 185 217]
 [  2 205  46  25  43 158  50 101]
 [141 135  18  12   5 253 210 176]
 [  6  91 134  77 191 122 232  33]
 [ 53  91   9  26   6 206  75  73]
 [153  69 169 187 204  42  30 242]
 [204  22 239  24   2 229 121  37]] 

quantized DCT coefficents: 
 [[434 -38  32   3 -90 -29  39   4]
 [  5  -7  13 -42 -50 -57 -60   1]
 [ 27  68 -15 -63  18   9  57  45]
 [-11  -2 -56 -29  20 -10 -40  23]
 [ 13   1 -16  31 -60  50   4  29]
 [  5   7   8 -19  58 -29  -3 -44]
 [-86  19 -11  41 -67 -13  10 -64]
 [  0  23 -82 -48  30 -25 -41  41]]


In [10]:
# Checking my work by comparing against Hany's solution in the slides
# min 15 on https://farid.berkeley.edu/learnComputerVision/lectures/cv-02-14-JPEGcompression.mp4

def hanySolution(B,Q):
    D = np.zeros((8,8)) # initialize
    y,x = np.meshgrid(np.arange(1,9,1), np.arange(1,9,1))
    for i in range(1,9):
        for j in range(1,9):
            ai = np.sqrt(1/8) if i == 1 else np.sqrt(2/8)
            aj = np.sqrt(1/8) if j == 1 else np.sqrt(2/8)
            D[i-1,j-1] = ai * aj * np.sum( np.sum(B * np.cos(np.pi*(2*x-1)*(i-1)/16) *
                                                      np.cos(np.pi*(2*y-1)*(j-1)/16)))
    return( np.array(D/Q, dtype=int) ) # quantize
                  
print('input pixel matrix: \n', P, '\n')
print('quantized DCT coefficients: \n', hanySolution(P,Q))

input pixel matrix: 
 [[  7 194 227  80 114  76  96  79]
 [108 160 202  20 104 107 185 217]
 [  2 205  46  25  43 158  50 101]
 [141 135  18  12   5 253 210 176]
 [  6  91 134  77 191 122 232  33]
 [ 53  91   9  26   6 206  75  73]
 [153  69 169 187 204  42  30 242]
 [204  22 239  24   2 229 121  37]] 

quantized DCT coefficients: 
 [[434 -38  32   3 -90 -29  39   4]
 [  5  -7  13 -42 -50 -57 -60   1]
 [ 27  68 -15 -63  18   9  57  45]
 [-11  -2 -56 -29  20 -10 -40  23]
 [ 13   1 -16  31 -60  50   4  29]
 [  5   7   8 -19  58 -29  -3 -44]
 [-86  19 -11  41 -67 -13  10 -64]
 [  0  23 -82 -48  30 -25 -41  41]]
